# Choosing representative sample set

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import random
%matplotlib notebook
# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"


# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [8]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

611

In [ ]:
#Use citypy to associate with the nearest city

cities_df["Closest City name"] = "" 
cities_df["Closest Country code"] = "" 

#for index,row in cities_df.iterrows():
    city = citipy.nearest_city(row["Latitude"],row["Longitude"])
    cities_df.set_value(index,"Closest City name",city.city_name)
    cities_df.set_value(index,"Closest Country code",city.country_code)

In [ ]:
# Dropping random lat and lon samples from dataframe as they are not the exact coordinates for the
#cities.

clean_cities_df = cities_df.drop(['Latitude', 'Longitude'],axis=1)
clean_cities_df

# Dropping duplicate cities which may have resulted in the previous step

clean_cities_df = clean_cities_df.drop_duplicates()
clean_cities_df.shape

In [ ]:
# Pick sample of 550 cities 
selected_cities = clean_cities_df.sample(550)

selected_cities = selected_cities.reset_index(drop=True)

# OpenWeatherMap

In [ ]:
#Setting values for OpenWeatherMap API Calls

base_url = "http://api.openweathermap.org/data/2.5/weather?"

app_id = api_key

params = {"appid" :app_id, "units":metric}



In [ ]:
# SAMPLE JSON RESPONSE -
#{"coord":{"lon":-0.13,"lat":51.51},
#"weather":[{"id":300,"main":"Drizzle","description":"light intensity drizzle","icon":"09d"}],
#"base":"stations",
#"main":{"temp":280.32,"pressure":1012,"humidity":81,"temp_min":279.15,"temp_max":281.15},
#"visibility":10000,
# "wind":{"speed":4.1,"deg":80},
# "clouds":{"all":90},
# "dt":1485789600,
# "sys":{"type":1,"id":5091,"message":0.0103,"country":"GB","sunrise":1485762037,"sunset":14857948
#75},
# "id":2643743,"name":"London","cod":200}

def encrypt_key(input_url):
    return input_url[0:53]+"<YourKey>"+input_url[85:]

for index,row in selected_cities.iterrows():
    params["q"] =f'{row["Closest City name"]},{row["Closest Country code"]}'
    print(f"Retrieving weather information for {params['q']}")
    city_weather_resp = requests.get(base_url,params)
    print(encrypt_key(city_weather_resp.url))
    city_weather_resp = city_weather_resp.json() 
    selected_cities.set_value(index,"Latitude",city_weather_resp.get("coord",{}).get("lat")) 
    selected_cities.set_value(index,"Longitude",city_weather_resp.get("coord",{}).get("lon")) 
    selected_cities.set_value(index,"Temperature",city_weather_resp.get("main",{}).get("temp_max"))

    selected_cities.set_value(index,"Wind speed",city_weather_resp.get("wind",{}).get("speed"))
    selected_cities.set_value(index,"Humidity",city_weather_resp.get("main",{}).get("humidity"))
    selected_cities.set_value(index,"Cloudiness",city_weather_resp.get("clouds",{}).get("all"))

In [ ]:
# Drop cities with missing information
selected_cities = selected_cities.dropna()
selected_cities.shape
selected_cities.to_csv("City_Weather_data.csv")

# Weather Analysis